# 🤖 RotterMaatje Fine-Tuning Notebook

This notebook fine-tunes a **Qwen3-4B-Instruct** model on RotterMaatje conversational data using:
1. **SFT (Supervised Fine-Tuning)** - Refining the empathetic conversation style
2. **DPO (Direct Preference Optimization)** - Aligning with safety and accuracy preferences

**Requirements:**
- Google Colab with GPU (T4 or better)
- Upload your data files when prompted

---

## 📦 Step 1: Install Dependencies

This will install Unsloth and all required packages. Takes ~2-3 minutes.

In [ ]:
%%capture
# Install Unsloth (optimized for Colab)
!pip install unsloth
# Install the latest compatible packages
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
# Verify GPU availability
import torch
print(f"🔧 PyTorch version: {torch.__version__}")
print(f"🎮 CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"📊 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ No GPU detected! Go to Runtime > Change runtime type > GPU")

## 📤 Step 2: Upload Your Data

You need to upload two files:
1. `synthetic_train.jsonl` - SFT training data
2. `synthetic_dpo.jsonl` - DPO preference data

In [ ]:
from google.colab import files
import os

print("📁 Please upload your training data files...")
print("   - synthetic_train.jsonl (SFT data)")
print("   - synthetic_dpo.jsonl (DPO data)")
print()

uploaded = files.upload()

for filename in uploaded.keys():
    print(f"✅ Uploaded: {filename} ({len(uploaded[filename]) / 1024:.1f} KB)")

In [ ]:
# Verify data files
import json

def count_lines(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        return sum(1 for _ in f)

if os.path.exists('synthetic_train.jsonl'):
    sft_count = count_lines('synthetic_train.jsonl')
    print(f"✅ SFT data: {sft_count} conversations")
else:
    print("❌ Missing: synthetic_train.jsonl")

if os.path.exists('synthetic_dpo.jsonl'):
    dpo_count = count_lines('synthetic_dpo.jsonl')
    print(f"✅ DPO data: {dpo_count} preference pairs")
else:
    print("❌ Missing: synthetic_dpo.jsonl")

## 🎓 Step 3: Supervised Fine-Tuning (SFT)

Refine the instruct model with RotterMaatje's specific persona and knowledge.

In [ ]:
from unsloth import FastLanguageModel
import torch

# Configuration
MODEL_NAME = "Qwen/Qwen3-4B-Instruct-2507"  # 4B model - fits on Colab T4
MAX_SEQ_LENGTH = 2048

print(f"🚀 Loading {MODEL_NAME}...")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=MAX_SEQ_LENGTH,
    load_in_4bit=True,
    dtype=None,  # Auto-detect
)

print("🎨 Adding LoRA adapters...")
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Rank
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",    
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
)

print("✅ Model loaded and adapters attached!")

In [ ]:
from unsloth.chat_templates import get_chat_template

# Apply chat template for Qwen
tokenizer = get_chat_template(
    tokenizer,
    chat_template="qwen-2.5",  # Compatible with latest Qwen models
)

print("✅ Chat template configured!")

In [ ]:
# Add LoRA adapters for efficient fine-tuning
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA rank
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
)

print("✅ LoRA adapters added!")

In [ ]:
from datasets import load_dataset

# Load SFT dataset
dataset = load_dataset("json", data_files="synthetic_train.jsonl", split="train")

def formatting_prompts_func(examples):
    """Format conversations into chat template format."""
    convos = examples["messages"]
    texts = [
        tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False)
        for convo in convos
    ]
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)

print(f"✅ Loaded {len(dataset)} training conversations")
print(f"\n📝 Sample formatted text:\n{dataset[0]['text'][:500]}...")

In [ ]:
from trl import SFTTrainer, SFTConfig

print("🎯 Starting SFT Training...")
print("   This will take ~10-20 minutes on a T4 GPU")

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    dataset_num_proc=2,
    packing=False,
    args=SFTConfig(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=100,  # Adjust based on dataset size
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=5,
        weight_decay=0.01,
        output_dir="outputs_sft",
        optim="adamw_8bit",
        seed=3407,
        report_to="none",
    ),
)

# Train!
trainer_stats = trainer.train()

print("\n✅ SFT Training complete!")
print(f"   Training loss: {trainer_stats.training_loss:.4f}")

In [ ]:
# Save SFT checkpoint for DPO
model.save_pretrained("lora_sft_checkpoint")
tokenizer.save_pretrained("lora_sft_checkpoint")
print("✅ SFT checkpoint saved!")

## 🎯 Step 4: Direct Preference Optimization (DPO)

Align the model with safety and accuracy preferences using chosen/rejected pairs.

In [ ]:
from unsloth import PatchDPOTrainer
from trl import DPOTrainer, DPOConfig
from datasets import load_dataset

# Patch DPO for Unsloth optimizations
PatchDPOTrainer()

# Load DPO dataset
dpo_dataset = load_dataset("json", data_files="synthetic_dpo.jsonl", split="train")

print(f"✅ Loaded {len(dpo_dataset)} DPO preference pairs")
print(f"\n📝 Sample DPO pair:")
print(f"   Prompt: {dpo_dataset[0]['prompt'][:100]}...")
print(f"   Chosen: {dpo_dataset[0]['chosen'][:100]}...")
print(f"   Rejected: {dpo_dataset[0]['rejected'][:100]}...")

In [ ]:
print("🎯 Starting DPO Training...")
print("   This will take ~10-15 minutes on a T4 GPU")

dpo_trainer = DPOTrainer(
    model=model,
    ref_model=None,  # Unsloth handles this for PEFT
    train_dataset=dpo_dataset,
    tokenizer=tokenizer,
    args=DPOConfig(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=8,
        warmup_steps=5,
        max_steps=60,
        learning_rate=5e-6,  # Lower LR for DPO
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=5,
        optim="adamw_8bit",
        weight_decay=0.05,
        output_dir="outputs_dpo",
        seed=3407,
        report_to="none",
        beta=0.1,  # DPO KL penalty
        max_length=1024,
        max_prompt_length=512,
    ),
)

# Train!
dpo_stats = dpo_trainer.train()

print("\n✅ DPO Training complete!")

## 💾 Step 5: Export Model

Export the fine-tuned model in GGUF format for use with llama.cpp or Ollama.

In [ ]:
print("📦 Exporting model to GGUF format...")
print("   This may take 5-10 minutes")

# Export as Q4_K_M quantization (good balance of size/quality)
model.save_pretrained_gguf(
    "rottermaatje_q4_k_m",
    tokenizer,
    quantization_method="q4_k_m"
)

print("\n✅ Model exported successfully!")

In [ ]:
# List exported files
import os

print("📁 Exported files:")
for f in os.listdir("rottermaatje_q4_k_m"):
    size = os.path.getsize(f"rottermaatje_q4_k_m/{f}") / 1e6
    print(f"   {f} ({size:.1f} MB)")

In [ ]:
# Download the GGUF file
from google.colab import files
import glob

gguf_files = glob.glob("rottermaatje_q4_k_m/*.gguf")
if gguf_files:
    print(f"⬇️ Downloading: {gguf_files[0]}")
    files.download(gguf_files[0])
else:
    print("❌ No GGUF file found!")

## 🧪 Step 6: Test the Model (Optional)

Quick test to verify the fine-tuned model works correctly.

In [ ]:
# Enable inference mode
FastLanguageModel.for_inference(model)

# Test prompts in different languages
test_prompts = [
    "Ik ben dakloos en heb honger. Waar kan ik eten krijgen?",
    "I see a homeless person on the street. What can I do to help?",
    "أين يمكنني العثور على مأوى ليلي في روتردام؟",
    "Gdzie mogę znaleźć nocleg w Rotterdamie?"
]

print("🧪 Testing fine-tuned model:\n")

for prompt in test_prompts:
    messages = [{"role": "user", "content": prompt}]
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")

    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=256,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract just the assistant response
    if "assistant" in response.lower():
        response = response.split("assistant")[-1].strip()

    print(f"📝 User: {prompt[:60]}...")
    print(f"🤖 RotterMaatje: {response[:200]}...")
    print("-" * 60)

## ✅ Done!

Your fine-tuned RotterMaatje model has been:
1. ✅ Trained with SFT on empathetic conversations
2. ✅ Aligned with DPO on safety preferences
3. ✅ Exported as GGUF for local deployment

### Next Steps:
1. Download the `.gguf` file from the outputs
2. Place it in your `Jack/models/` directory
3. Configure Ollama or llama.cpp to use the model

### Ollama Setup:
```bash
# Create a Modelfile
echo 'FROM ./rottermaatje_q4_k_m.gguf' > Modelfile

# Create the model in Ollama
ollama create rottermaatje -f Modelfile

# Test it
ollama run rottermaatje
```